## Multi Step Binomial Option Pricing

Stock tree is represented using nodes, using (i,j) coordinates, where i is time period and j is upward movement, and S0 is the orginal stock price.

$ S0_{(i,j)} = S0 * u^j * d^{i-j} $




#### Method 1
1. create forward pass for the stock price 
2. calculate the payoff 
3. backward pass to calculate option price at C[0]

In [1]:
import numpy as np
import math

In [2]:
S0=100
K=100
T=1 
u=1.1
d=1/u 
N=5
r=0.06

In [3]:
# multi step binomial option pricing 

def multi_step_option_pricing(S0, K, T, u, d, N, r, objecttype='C'):
# pre compute the variables 
    dt = T/N
    p = (np.exp(r*dt) - d) / (u - d)  # risk neutral probability
    disc = np.exp(-r*dt)              # disc factor 
    
# forward pass for stock price 
    S = np.zeros(N+1)
    S[0] = S0 * d**N 
    for j in range(1, N+1):
        S[j] = S0 * u**j * d**(N-j)
        
# calculate the payoff 
    C = np.zeros(N+1)
    for j in range(0, N+1):
        C[j] = max((S[j] - K), 0)

# backward pass for option price using discouting 
    for i in range(N, 0, -1):    # loop backward from the last time step 
        for j in range(0, i):      # for each time step iterate over each time step 
            C[j] = disc * ( p * C[j+1] + (1 - p) * C[j])

    return C[0]
    

In [4]:
multi_step_option_pricing(S0, K, T, u, d, N, r, 'C')

np.float64(11.839823325808702)

In [23]:
S0=100
K=100
T=1 
u=1.1
d=1/u 
N=50
r=0.06
objecttype='C'

### Vectorized

In [24]:
# multi step binomial option pricing 
def binomial_option_pricing_vectorized(S0, K, T, d, u,  N, r, objecttype='C'):
# pre compute the variables 
    dt = T/N
    p = (np.exp(r*dt) - d) / (u - d)  # risk neutral probability
    disc = np.exp(-r*dt)              # disc factor 
    
# forward pass for stock price 
    j = np.arange(N+1)
    S = S0 * u**j * d**(N-j)
        
# calculate the payoff 
    C = np.maximum((S - K), 0)

# backward pass for option price using discouting 
    for i in range(N, 0, -1):    # loop backward from the last time step  
            C = disc * ( p * C[1:i+1] + (1 - p) * C[:i])   # vectorize using slice the last call value row
            print(f"time: {i}, option value :{C}")

    return C[0]

In [25]:
binomial_option_pricing_vectorized(S0, K, T, d, u, N, r, objecttype)

time: 50, option value :[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.47858388e-14 1.01199280e+01 3.32199280e+01 6.11709280e+01
 9.49916380e+01 1.35914697e+02 1.85431599e+02 2.45347049e+02
 3.17844745e+02 4.05566957e+02 5.11710833e+02 6.40144922e+02
 7.95550171e+02 9.83590522e+02 1.21111935e+03 1.48642923e+03
 1.81955418e+03 2.22263537e+03 2.71036361e+03 3.30051479e+03
 4.01459771e+03 4.87863804e+03 5.92412684e+03 7.18916830e+03
 8.71986845e+03 1.05720156e+04]
time: 49, option value :[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00

np.float64(28.49926835944145)

### CRR Model 

CRR introduce: 
$$ u = e^{r \sqrt \Delta t} $$
$$ d = e^{- r \sqrt \Delta t}  = 1/u $$
 

In [ ]:
# multi step binomial option pricing 
def crr_binomial_option(S0, K, T, N, r, sigma, objecttype='C'):
# pre compute the variables
    dt = T/N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1/u
    p =  (np.exp(r*dt) - d) / (u - d) #risk neutral probability
    
    disc = np.exp(-r*dt)              # disc factor 
    
# forward pass for stock price 
    j = np.arange(N+1)
    S = S0 * u**j * d**(N-j)
    
    if objecttype == 'C':
        C = np.maximum(S - K, 0)
    else:
        C = np.maximum(K - S, 0)

# backward pass for option price using discouting 
    for i in range(N, 0, -1):    # loop backward from the last time step  
            C = disc * ( p * C[1:i+1] + (1 - p) * C[:i])   # vectorize using slice on the row
            print(f"time: {i}, option value :{C}")
                  
    return C[0]

In [45]:
crr_binomial_option(100, 100, 1, 50, 0.06, 0.02, objecttype='C')

time: 50, option value :[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.40317112  0.97206637  1.5441889   2.11955701  2.69818912
  3.28010375  3.8653195   4.45385512  5.04572944  5.64096139  6.23957002
  6.84157448  7.44699405  8.0558481   8.6681561   9.28393766  9.90321247
 10.52600036 11.15232125 11.78219519 12.41564233 13.05268295 13.69333742
 14.33762625 14.98557005]
time: 49, option value :[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.28653316  0.80700068  1.37750728  1.95125031  2.52824811  3.10851916
  3.69208202  4.27895536  4.86915796  5.4

np.float64(5.823932030383528)